In [1]:
import pandas as pd
from collections import defaultdict
import warnings
import visualize
import neat
import multiprocessing
import os
import matplotlib.pyplot as plt

import numpy as np

In [5]:
necoc = 30
generations = int(3000/necoc)
generations

100

In [2]:
def softmax(x, axis=1):
    # 计算每行的最大值
    row_max = x.max(axis=axis)
 
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

In [27]:
df = pd.read_csv('letter-recognition.data',header=None)

train_df = df[:1600]
test_df = df[16000:16000+400]

X = train_df.ix[:,1:16].to_numpy() 
y = np.array([ord(l)-ord('A') for l in train_df.ix[:,0].to_numpy()])

testX = test_df.ix[:,1:16].to_numpy() 
testy = np.array([ord(l)-ord('A') for l in test_df.ix[:,0].to_numpy()])

In [23]:
def get_winner(class1):
    
    numbers = [*range(26)] # number 2 > number 1
    number_of_classification = len(numbers)
    #number_of_sampling = 100

    list_y = y.tolist()
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

        
    class2 = []

    class1_indexes = []
    class2_indexes = []

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)

    print("number of class1:{}".format(len(class1_label)))
    print("number of class2:{}".format(len(class2_label)))

    samplesize = 1600
    x_inputs =[tuple(c) for c in X[class1_indexes].tolist()] + [tuple(c) for c in X[class2_indexes].tolist()]
    x_outputs = [tuple([c]) for c in class1_label + class2_label]
    
    
    def eval_genomes(genomes, config):
        for genome_id, genome in genomes:
            net = neat.nn.FeedForwardNetwork.create(genome, config)

            outputs = []
            for xi in x_inputs:
                output = net.activate(xi)
                outputs.append(output)


            px_outputs = softmax(np.array(outputs).reshape(samplesize, 2), axis=1)
            # the index of maximum in each line
            pred_outputs = np.argmax(px_outputs, axis = 1)
            real_outputs = np.array(x_outputs).reshape(samplesize,)

            acc = np.sum(pred_outputs == real_outputs)/samplesize

            genome.fitness = acc

    def run(config_file):
        # Load configuration.
        config = neat.Config(
            neat.DefaultGenome,
            neat.DefaultReproduction,
            neat.DefaultSpeciesSet,
            neat.DefaultStagnation,
            config_file,
        )

        # Create the population, which is the top-level object for a NEAT run.
        p = neat.Population(config)

        # add a stdout reporter to show progress in the terminal
        reporter = neat.StdOutReporter(False)
        p.add_reporter(reporter)
        stats = neat.StatisticsReporter()
        p.add_reporter(stats)
        #checkpointer = neat.Checkpointer(100)
        #p.add_reporter(checkpointer)
        # Run for up to 300 generations.
        winner = p.run(eval_genomes, generations)

        return [stats, winner]
    
    local_dir = os.getcwd()
    config_path = os.path.join(local_dir, "config-feedforward-letter2")
    [stats, winner] = run(config_path)
    
    return winner

In [14]:
import numpy as np
import random

number_codes = []
for i in range(2**(26-1), 2**26-1):
    if (i % 1000000) == 0:
            print(i)
    for j in bin(i).split('b')[1]:
        number_codes.append(int(j))
        


number_codes = np.array(number_codes).reshape(2**(26 - 1) - 1,26).T     

slice = random.sample(range(2**(26 - 1) - 1), necoc)

number_codes = number_codes[:,slice].tolist()

classes = []
for i in range(necoc):
    code_list = np.array(number_codes).T[i]
    classes.append(np.where(code_list==1)[0].tolist())

matrix_errors = []

for i in range(25):
    for j in range(i+1,26):
        matrix_errors.append(necoc - np.sum(np.array(number_codes[i]) == np.array(number_codes[j])))
min_error = np.min(matrix_errors)

print(min_error)

34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000
60000000
61000000
62000000
63000000
64000000
65000000
66000000
67000000
4


30

In [24]:
winner_list = []

for i,class1 in enumerate(classes):
    print("running:{}/30".format(i))
    winner = get_winner(class1)
    winner_list.append(winner)

running:0/30
number of class1:671
number of class2:929

 ****** Running generation 0 ****** 

Population's average fitness: 0.50940 stdev: 0.07099
Best fitness: 0.59750 - size: (2, 3) - species 4 - id 17
Average adjusted fitness: 0.108
Mean genetic distance 2.549, standard deviation 0.429
Population of 201 members in 10 species
Total extinctions: 0
Generation time: 2.804 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.54056 stdev: 0.06390
Best fitness: 0.60313 - size: (3, 5) - species 6 - id 319
Average adjusted fitness: 0.137
Mean genetic distance 2.614, standard deviation 0.482
Population of 201 members in 12 species
Total extinctions: 0
Generation time: 3.003 sec (2.903 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.54090 stdev: 0.06512
Best fitness: 0.60750 - size: (3, 5) - species 7 - id 433
Average adjusted fitness: 0.134
Mean genetic distance 2.635, standard deviation 0.498
Population of 199 members in 13 species
Total 

Generation time: 5.054 sec (4.986 average)

 ****** Running generation 24 ****** 

Population's average fitness: 0.55456 stdev: 0.06571
Best fitness: 0.63062 - size: (6, 13) - species 30 - id 2237
Average adjusted fitness: 0.140
Mean genetic distance 3.107, standard deviation 0.584
Population of 192 members in 22 species
Total extinctions: 0
Generation time: 5.072 sec (4.960 average)

 ****** Running generation 25 ****** 

Population's average fitness: 0.55581 stdev: 0.06564
Best fitness: 0.63062 - size: (6, 13) - species 30 - id 2237
Average adjusted fitness: 0.140
Mean genetic distance 3.088, standard deviation 0.593
Population of 206 members in 20 species
Total extinctions: 0
Generation time: 5.287 sec (5.023 average)

 ****** Running generation 26 ****** 

Population's average fitness: 0.55392 stdev: 0.06234
Best fitness: 0.63062 - size: (6, 13) - species 30 - id 2237
Average adjusted fitness: 0.140
Mean genetic distance 3.115, standard deviation 0.584
Population of 205 members in 

Population of 199 members in 13 species
Total extinctions: 0
Generation time: 7.395 sec (7.154 average)

 ****** Running generation 72 ****** 

Population's average fitness: 0.53693 stdev: 0.07366
Best fitness: 0.64750 - size: (7, 16) - species 31 - id 10311
Average adjusted fitness: 0.124
Mean genetic distance 3.186, standard deviation 0.612
Population of 198 members in 13 species
Total extinctions: 0
Generation time: 7.453 sec (7.259 average)

 ****** Running generation 73 ****** 

Population's average fitness: 0.54173 stdev: 0.07366
Best fitness: 0.64750 - size: (7, 16) - species 31 - id 10311
Average adjusted fitness: 0.147
Mean genetic distance 3.161, standard deviation 0.630
Population of 198 members in 13 species
Total extinctions: 0
Generation time: 7.229 sec (7.156 average)

 ****** Running generation 74 ****** 

Population's average fitness: 0.54205 stdev: 0.07431
Best fitness: 0.64750 - size: (7, 16) - species 31 - id 10311
Average adjusted fitness: 0.124
Mean genetic distan

Mean genetic distance 3.115, standard deviation 0.579
Population of 197 members in 15 species
Total extinctions: 0
Generation time: 8.937 sec (9.488 average)

 ****** Running generation 96 ****** 

Population's average fitness: 0.55938 stdev: 0.06855
Best fitness: 0.64812 - size: (12, 24) - species 74 - id 14384
Average adjusted fitness: 0.147
Mean genetic distance 3.125, standard deviation 0.563
Population of 195 members in 14 species
Total extinctions: 0
Generation time: 8.765 sec (9.042 average)

 ****** Running generation 97 ****** 

Population's average fitness: 0.56020 stdev: 0.06655
Best fitness: 0.64812 - size: (12, 24) - species 74 - id 14384
Average adjusted fitness: 0.145
Mean genetic distance 3.111, standard deviation 0.553
Population of 204 members in 14 species
Total extinctions: 0
Generation time: 9.159 sec (9.099 average)

 ****** Running generation 98 ****** 

Population's average fitness: 0.56806 stdev: 0.05981
Best fitness: 0.64812 - size: (12, 24) - species 74 - id 


 ****** Running generation 20 ****** 

Population's average fitness: 0.56993 stdev: 0.11141
Best fitness: 0.67937 - size: (5, 12) - species 17 - id 2730
Average adjusted fitness: 0.204
Mean genetic distance 3.149, standard deviation 0.567
Population of 189 members in 33 species
Total extinctions: 0
Generation time: 4.776 sec (4.559 average)

 ****** Running generation 21 ****** 

Population's average fitness: 0.57334 stdev: 0.10703
Best fitness: 0.68375 - size: (3, 4) - species 42 - id 3005
Average adjusted fitness: 0.244
Mean genetic distance 3.170, standard deviation 0.549
Population of 190 members in 31 species
Total extinctions: 0
Generation time: 4.551 sec (4.560 average)

 ****** Running generation 22 ****** 

Population's average fitness: 0.56849 stdev: 0.11184
Best fitness: 0.68375 - size: (3, 4) - species 42 - id 3005
Average adjusted fitness: 0.236
Mean genetic distance 3.212, standard deviation 0.561
Population of 207 members in 28 species
Total extinctions: 0
Generation ti

...

Population's average fitness: 0.56338 stdev: 0.08654
Best fitness: 0.66438 - size: (6, 15) - species 55 - id 11723
Average adjusted fitness: 0.172
Mean genetic distance 3.058, standard deviation 0.615
Population of 203 members in 13 species
Total extinctions: 0
Generation time: 6.878 sec (6.873 average)

 ****** Running generation 97 ****** 

Population's average fitness: 0.55874 stdev: 0.09000
Best fitness: 0.66438 - size: (6, 15) - species 55 - id 11723
Average adjusted fitness: 0.172
Mean genetic distance 3.065, standard deviation 0.613
Population of 201 members in 14 species
Total extinctions: 0
Generation time: 7.032 sec (6.915 average)

 ****** Running generation 98 ****** 

Population's average fitness: 0.55414 stdev: 0.09178
Best fitness: 0.66438 - size: (6, 15) - species 55 - id 11723
Average adjusted fitness: 0.165
Mean genetic distance 2.934, standard deviation 0.553
Population of 200 members in 12 species
Total extinctions: 0
Generation time: 7.536 sec (6.983 average)

 ***

Population's average fitness: 0.53061 stdev: 0.03844
Best fitness: 0.63187 - size: (6, 11) - species 21 - id 3028
Average adjusted fitness: 0.072
Mean genetic distance 3.258, standard deviation 0.520
Population of 201 members in 34 species
Total extinctions: 0
Generation time: 4.396 sec (4.216 average)

 ****** Running generation 21 ****** 

Population's average fitness: 0.53402 stdev: 0.03812
Best fitness: 0.63187 - size: (6, 11) - species 21 - id 3028
Average adjusted fitness: 0.046
Mean genetic distance 3.232, standard deviation 0.543
Population of 192 members in 31 species
Total extinctions: 0
Generation time: 4.518 sec (4.265 average)

 ****** Running generation 22 ****** 

Population's average fitness: 0.53109 stdev: 0.03703
Best fitness: 0.63187 - size: (6, 11) - species 21 - id 3028
Average adjusted fitness: 0.057
Mean genetic distance 3.254, standard deviation 0.534
Population of 197 members in 33 species
Total extinctions: 0
Generation time: 4.273 sec (4.270 average)

 ******

Population's average fitness: 0.53905 stdev: 0.04469
Best fitness: 0.64062 - size: (7, 12) - species 21 - id 5859
Average adjusted fitness: 0.093
Mean genetic distance 3.165, standard deviation 0.567
Population of 196 members in 21 species
Total extinctions: 0
Generation time: 5.319 sec (5.108 average)

 ****** Running generation 45 ****** 

Population's average fitness: 0.53927 stdev: 0.04462
Best fitness: 0.64062 - size: (7, 12) - species 21 - id 5859
Average adjusted fitness: 0.064
Mean genetic distance 3.126, standard deviation 0.577
Population of 196 members in 20 species
Total extinctions: 0
Generation time: 5.326 sec (5.158 average)

 ****** Running generation 46 ****** 

Population's average fitness: 0.54062 stdev: 0.04452
Best fitness: 0.64062 - size: (7, 12) - species 21 - id 5859
Average adjusted fitness: 0.081
Mean genetic distance 3.157, standard deviation 0.566
Population of 198 members in 20 species
Total extinctions: 0
Generation time: 5.263 sec (5.189 average)

 ******

Generation time: 5.738 sec (5.824 average)

 ****** Running generation 69 ****** 

Population's average fitness: 0.54821 stdev: 0.04701
Best fitness: 0.64062 - size: (7, 12) - species 21 - id 5859
Average adjusted fitness: 0.094
Mean genetic distance 3.231, standard deviation 0.543
Population of 195 members in 22 species
Total extinctions: 0
Generation time: 5.804 sec (5.811 average)

 ****** Running generation 70 ****** 

Population's average fitness: 0.54752 stdev: 0.04745
Best fitness: 0.64062 - size: (7, 12) - species 21 - id 5859
Average adjusted fitness: 0.056
Mean genetic distance 3.242, standard deviation 0.543
Population of 200 members in 25 species
Total extinctions: 0
Generation time: 5.783 sec (5.797 average)

 ****** Running generation 71 ****** 

Population's average fitness: 0.54878 stdev: 0.04832
Best fitness: 0.64062 - size: (7, 12) - species 21 - id 5859
Average adjusted fitness: 0.063
Mean genetic distance 3.237, standard deviation 0.563
Population of 197 members in 

Population of 201 members in 16 species
Total extinctions: 0
Generation time: 6.671 sec (6.516 average)

 ****** Running generation 93 ****** 

Population's average fitness: 0.54921 stdev: 0.04784
Best fitness: 0.64375 - size: (8, 19) - species 49 - id 14158
Average adjusted fitness: 0.107
Mean genetic distance 3.062, standard deviation 0.532
Population of 200 members in 15 species
Total extinctions: 0
Generation time: 6.674 sec (6.544 average)

 ****** Running generation 94 ****** 

Population's average fitness: 0.54930 stdev: 0.04877
Best fitness: 0.64375 - size: (8, 19) - species 49 - id 14158
Average adjusted fitness: 0.078
Mean genetic distance 3.071, standard deviation 0.539
Population of 195 members in 15 species
Total extinctions: 0
Generation time: 6.634 sec (6.546 average)

 ****** Running generation 95 ****** 

Population's average fitness: 0.55412 stdev: 0.04764
Best fitness: 0.64375 - size: (8, 19) - species 49 - id 14158
Average adjusted fitness: 0.115
Mean genetic distan

In [30]:
local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-letter2")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

def get_pred_real(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = testX[j]
    test_y = testy[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

from random import randint
pred_value = []
error_list = []

for j in range(400):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_real(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(necoc - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )
list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
np.sum(list_P == testy)/400

0.3575

In [33]:
def get_pred_train(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = X[j]
    test_y = y[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

pred_value = []
error_list = []

for j in range(1600):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_train(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(26 - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )

list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
print("Trainset acc:{}".format(np.sum(list_P == y)/1600))
winner_fitness = []
for winner in winner_list:
    #print(winner.fitness)
    winner_fitness.append(winner.fitness)
print("Avg Base Learner:{}".format(np.mean(winner_fitness)))

Trainset acc:0.4025
Avg Base Learner:0.6972916666666668


In [34]:
# program to check if there is exist a path between two vertices 
# of a graph 

from collections import defaultdict 

#This class represents a directed graph using adjacency list representation 
class Graph: 

	def __init__(self,vertices): 
		self.V= vertices #No. of vertices 
		self.graph = defaultdict(list) # default dictionary to store graph 

	# function to add an edge to graph 
	def addEdge(self,u,v): 
		self.graph[u].append(v) 
	
	# Use BFS to check path between s and d 
	def isReachable(self, s, d): 
		# Mark all the vertices as not visited 
		visited =[False]*(self.V) 

		# Create a queue for BFS 
		queue=[] 

		# Mark the source node as visited and enqueue it 
		queue.append(s) 
		visited[s] = True

		while queue: 

			#Dequeue a vertex from queue 
			n = queue.pop(0) 
			
			# If this adjacent node is the destination node, 
			# then return true 
			if n == d: 
				return True

			# Else, continue to do BFS 
			for i in self.graph[n]: 
				if visited[i] == False: 
					queue.append(i) 
					visited[i] = True
		# If BFS is complete without visited d 
		return False
    
    
def findAllPath(graph,start,end,path=[]):
    path = path +[start]
    if start == end:
        return [path]
 
    paths = [] #存储所有路径    
    for node in graph[start]:
        if node not in path:
            newpaths = findAllPath(graph,node,end,path) 
            for newpath in newpaths:
                paths.append(newpath)
    return paths

numbers = [*range(2)]
number_of_classification = len(numbers)

list_nodes_number = []
list_connection_number = []

for winner in winner_list:
    used_nodes = list(winner.nodes.keys())
    # create a dict for mapping 
    l1 = numbers + config.genome_config.input_keys + used_nodes[number_of_classification:]
    l2 = range(len(l1))
    dict_nodes = dict(zip(l1, l2))

    ### 将节点加入图
    g = Graph(len(l1))# inputs + outputs + used  
    # add connections in the graph
    for cg in winner.connections.values():
        if cg.enabled:
            g.addEdge(dict_nodes[cg.key[0]],dict_nodes[cg.key[1]])

    v = numbers
    list_connections = []
    for vi in v:
        for u in range(number_of_classification, len(config.genome_config.input_keys) + number_of_classification):
            list_connections.append(g.isReachable(u, vi))

    nodes_inputs = range(number_of_classification, number_of_classification+ 64)
    nodes_outputs = range(number_of_classification)
    nodes_mid = range(number_of_classification + 64, len(dict_nodes))

    # 找到所有输入和输出连接的路
    all_path = []
    for u in nodes_inputs:
        for v in nodes_outputs:
            path = findAllPath(g.graph, u, v)
            if path:
                all_path = all_path + path

    # 得到最长路，以及每个节点在哪个层级
    max_length = max([len(x) for x in all_path])
    nodes_tuples_list = []
    for path in all_path:
        #print(path)
        for node in path:
            nodes_tuples_list.append([node, path.index(node)])

    # 确定节点的层级（消除重复）       
    nodes_tuples_fixed_list = []
    for index in range(max_length):
        for nodes in nodes_tuples_list:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                if nodes not in nodes_tuples_fixed_list:
                    nodes_tuples_fixed_list.append(nodes)


    # 所有有连接的节点
    all_used_nodes = []
    for node in nodes_tuples_fixed_list:
        if node[0] not in all_used_nodes:
            all_used_nodes.append(node[0])

    # 确定节点的层级（最终确定）
    nodes_tuples_fixed_list_final = []

    for node in all_used_nodes:
        if node in nodes_inputs:
            nodes_tuples_fixed_list_final.append([node, 0])
        elif node in nodes_outputs:
            nodes_tuples_fixed_list_final.append([node, max_length - 1])
        else:
            list_node = []
            for nodes in nodes_tuples_fixed_list:
                if node == nodes[0]:
                    list_node.append(nodes)
            layer = 0
            for ele in list_node:
                # 删除重复项，取最大
                if ele[1] > layer:
                    layer = ele[1]
            nodes_tuples_fixed_list_final.append([node,layer])


    # 每个层级的节点合数
    length_of_layers = []
    number_of_nodes = 0
    for index in range(max_length):
        for nodes in nodes_tuples_fixed_list_final:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                number_of_nodes += 1
        length_of_layers.append(number_of_nodes)

    length_of_each_layer = []
    length_of_each_layer.append(length_of_layers[0])
    for i in range(1, max_length):
        length_of_each_layer.append(length_of_layers[i] - length_of_layers[i-1])

    # 输出每个层级的节点个数
    #print("length of each layers:", length_of_each_layer)

    # 所有端到端的路
    all_path_side2side = []
    for path in all_path:
        if len(path) == 2:
            all_path_side2side.append(path)
        else:
            for i in range(len(path)-1):
                all_path_side2side.append([path[i],path[i+1]])

    # 定义节点到绘图的字典
    dict_nodes_graph = dict()
    count_number_layer = [0] * max_length
    for nodes in nodes_tuples_fixed_list_final:
        node, layer = nodes
        dict_nodes_graph[node] = [count_number_layer[layer], layer]
        count_number_layer[layer] += 1
    list_nodes_number.append(np.sum(length_of_each_layer))
    list_connection_number.append(len(all_path_side2side))
    #print("Number of nodes:{} Number of connections:{}".format(np.sum(length_of_each_layer),len(all_path_side2side)))
print("Total nodes:{} Total connections::{}".format(np.sum(list_nodes_number), np.sum(list_connection_number)))

Total nodes:481 Total connections::2295
